In [1]:
# 查看当前挂载的数据集目录
!ls /home/kesci/input/TibetanMNIST5610/

TibetanMNIST  TibetanMNIST.npz	TibetanMNIST.tfrecords


In [2]:
# 查看个人持久化工作区文件
!ls /home/kesci/work/

lost+found


In [3]:
# 查看当前kernerl下的package
# !pip list --format=columns

In [4]:
# 显示cell运行时长
%load_ext klab-autotime

In [5]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.models import load_model
import pandas as pd
import numpy as np

Using TensorFlow backend.


time: 1.48 s


In [6]:
img_rows, img_cols = 28, 28

data = np.load('/home/kesci/input/TibetanMNIST5610/TibetanMNIST.npz')

image_data, label_data = data['image'], data['label']

print('image：', image_data.shape)
print('label：', label_data.shape)


image： (17768, 28, 28)
label： (17768,)
time: 108 ms


In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(image_data, label_data, test_size=0.2, random_state=42)

time: 391 ms


In [8]:
train_count = [0] * 10
test_count = [0] * 10
for tr_l in Y_train:
    train_count[tr_l] += 1
for te_l in Y_test:
    test_count[te_l] += 1
    
print('train_size: {}'.format(train_count))
print('train_size: {}'.format(test_count))

train_size: [1618, 1686, 1513, 1448, 1579, 1058, 1295, 1059, 1276, 1682]
train_size: [378, 448, 359, 368, 388, 292, 323, 243, 366, 389]
time: 5.85 ms


In [9]:
# Reshape image , Standardize , One-hot labels
img_rows, img_cols = 28, 28
input_shape = (img_rows, img_cols, 1) #tensorflow channels_last
num_classes = 10

time: 819 µs


In [10]:
X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1).astype('float32')/255
Y_train = keras.utils.to_categorical(Y_train, num_classes)

time: 44.7 ms


In [11]:
# Train model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))



model.compile(loss=keras.losses.categorical_crossentropy, 
              optimizer=keras.optimizers.Adadelta(), 
              metrics=['accuracy'])

time: 151 ms


In [12]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
__________

In [13]:
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1).astype('float32')/255
Y_test_onehot = keras.utils.to_categorical(Y_test, num_classes)

time: 48.6 ms


In [14]:
history = model.fit(X_train, Y_train, 
                    batch_size=128, epochs=25, 
                    verbose=1, 
                    validation_data=(X_test, Y_test_onehot))

Train on 14214 samples, validate on 3554 samples
Epoch 1/25
14214/14214 [==============================] - 38s 3ms/step - loss: 0.6187 - acc: 0.7906 - val_loss: 0.3347 - val_acc: 0.8858
Epoch 2/25
14214/14214 [==============================] - 37s 3ms/step - loss: 0.2335 - acc: 0.9229 - val_loss: 0.1310 - val_acc: 0.9530
Epoch 3/25
14214/14214 [==============================] - 37s 3ms/step - loss: 0.1592 - acc: 0.9471 - val_loss: 0.0868 - val_acc: 0.9699
Epoch 4/25
14214/14214 [==============================] - 38s 3ms/step - loss: 0.1232 - acc: 0.9568 - val_loss: 0.1509 - val_acc: 0.9477
Epoch 5/25
14214/14214 [==============================] - 37s 3ms/step - loss: 0.1042 - acc: 0.9660 - val_loss: 0.0749 - val_acc: 0.9733
Epoch 6/25
14214/14214 [==============================] - 37s 3ms/step - loss: 0.0886 - acc: 0.9709 - val_loss: 0.0587 - val_acc: 0.9814
Epoch 7/25
14214/14214 [==============================] - 37s 3ms/step - loss: 0.0810 - acc: 0.9719 - val_loss: 0.0678 - val_acc:

In [15]:
Y_predict = model.predict_classes(X_test)

time: 2.95 s


In [16]:
np.sum(Y_predict == Y_test)/len(Y_test)

0.9853685987619584

time: 9.64 ms


In [17]:
print(X_test.shape)
print(Y_test.shape)
print(Y_predict.shape)

(3554, 28, 28, 1)
(3554,)
(3554,)
time: 704 µs


In [18]:
import matplotlib.pyplot as plt

time: 123 ms


In [19]:
figure = plt.figure(figsize=(9, 8))
for i in range(9):
    plt.subplot(3, 3, i+1)
    plt.imshow(X_test[i,:,:].reshape(28,28))
    plt.title('识别结果是:{}, 实际是:{}'.format(Y_predict[i], Y_test[i]))
    plt.axis('off')
plt.show()

<Figure size 648x576 with 9 Axes>

time: 579 ms
